In [1]:
"""
This file contains the code to run the baseline experiments.

More specifically, we are investigating the performance of basic models on the CIFAR-10 and CIFAR-10H datasets. The tasks for these datasets are multi-class classification.

The basic models include:
    * ResNet-50
    * VGG-16
    * Logistic Regression
    * Random Forest
    * XGBoost
"""

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import os
from typing import Tuple

# Loading Data

In [2]:
# Load CIFAR-10H dataset and return a Dataset
def load_cifar10h() -> Dataset:
    cifar10h_probs_path = "../data/cifar-10h/cifar10h-probs.npy"
    if not os.path.exists(cifar10h_probs_path):
        raise FileNotFoundError(
            f"Soft labels not found at {cifar10h_probs_path}. Please ensure the CIFAR-10H data is downloaded."
        )

    cifar10h_probs = np.load(cifar10h_probs_path).astype(np.float32)
    cifar10_test = datasets.CIFAR10(
        root="../data/cifar-10", train=False, download=True, transform=transforms.ToTensor()
    )

    class CIFAR10H(Dataset):
        def __init__(self, cifar10_dataset: Dataset, soft_labels: np.ndarray):
            self.cifar10_dataset = cifar10_dataset
            self.soft_labels = soft_labels

        def __len__(self) -> int:
            return len(self.cifar10_dataset)

        def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
            image, _ = self.cifar10_dataset[idx]
            soft_label = torch.from_numpy(self.soft_labels[idx])
            return image.float(), soft_label

    cifar10h_dataset = CIFAR10H(cifar10_test, cifar10h_probs)
    return cifar10h_dataset

In [3]:
# Load CIFAR-10 dataset and return train, validation, and test DataLoaders
def load_cifar10() -> Tuple[Dataset, Dataset, Dataset]:
    transform = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.ConvertImageDtype(torch.float32),
        ]
    )
    full_dataset = datasets.CIFAR10(root="../data/cifar-10", train=True, download=True, transform=transform)

    # This dataset will be used for testing and validation.
    #   30% of the data will be used for validation, and 70% for testing.
    test_size = int(0.7 * len(full_dataset))
    val_size = len(full_dataset) - test_size
    test_dataset, val_dataset = torch.utils.data.random_split(
        full_dataset, [test_size, val_size], generator=torch.Generator().manual_seed(229)
    )

    return test_dataset, val_dataset

In [4]:
def get_ml_data(cifar10h_dataset, cifar10_test_dataset):
    # Prepare data for ML models
    X_cifar10h = np.array([img.numpy().flatten() for img, _ in cifar10h_dataset])
    y_cifar10h = np.array([np.argmax(label) for _, label in cifar10h_dataset])

    X_cifar10_test = np.array([img.numpy().flatten() for img, _ in cifar10_test_dataset])
    y_cifar10_test = np.array([label for _, label in cifar10_test_dataset])

    # Scale the data for ML models
    scaler = StandardScaler()
    X_cifar10h_scaled = scaler.fit_transform(X_cifar10h)
    X_cifar10_scaled_test = scaler.transform(X_cifar10_test)

    return X_cifar10h_scaled, y_cifar10h, X_cifar10_scaled_test, y_cifar10_test

In [5]:
cifar10h_dataset = load_cifar10h()
cifar10h_loader = DataLoader(cifar10h_dataset, batch_size=128, shuffle=True)
print(f"CIFAR-10H dataset loaded with {len(cifar10h_dataset)} samples")

cifar10_test_dataset, cifar10_val_dataset = load_cifar10()  # Changed variable name to reflect split
cifar10_test_loader = DataLoader(cifar10_test_dataset, batch_size=128, shuffle=False)
cifar10_val_loader = DataLoader(cifar10_val_dataset, batch_size=128, shuffle=False)
print(
    f"CIFAR-10 dataset loaded with {len(cifar10_test_dataset)} test and {len(cifar10_val_dataset)} validation samples"
)
X_cifar10h, y_cifar10h, X_cifar10_test, y_cifar10_test = get_ml_data(cifar10h_dataset, cifar10_test_dataset)

Files already downloaded and verified
CIFAR-10H dataset loaded with 10000 samples
Files already downloaded and verified
CIFAR-10 dataset loaded with 35000 test and 15000 validation samples


# Training
Training is done on the CIFAR-10H dataset. Evaluation is done on the CIFAR-10 train set, which we use as a test set.

In [6]:
def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    criterion: nn.Module,
    optimizer: optim.Optimizer,
    num_epochs: int,
) -> nn.Module:
    device = torch.device(
        "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
    )
    print(f"Using device: {device}")
    model = model.to(device)

    best_val_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Validation phase
        model.eval()
        correct = 0
        total = 0
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)

                if len(labels.shape) > 1:  # For soft labels
                    _, predicted = torch.max(outputs.data, 1)
                    _, labels = torch.max(labels, 1)
                else:  # For hard labels
                    _, predicted = torch.max(outputs.data, 1)

                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                val_loss += criterion(outputs, labels).item()

        accuracy = 100 * correct / total
        val_loss = val_loss / len(val_loader)
        print(
            f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {running_loss/len(train_loader):.4f}, Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%"
        )

        # Save model if validation accuracy improves
        if accuracy > best_val_acc:
            best_val_acc = accuracy
            torch.save(model.state_dict(), f"models/{model.__class__.__name__}_cifar10h.pth")
            print(f"Saved model with improved validation accuracy: {accuracy:.2f}%")

    return model

## Training Neural Networks

In [7]:
def train_nn_model(
    model, cifar10h_loader: DataLoader, cifar10_val_loader: DataLoader, num_epochs: int = 20, lr: float = 0.001
) -> list:
    print(f"\nTraining {model.__class__.__name__} on CIFAR-10H...")

    # Adjust the final layer for CIFAR-10
    if isinstance(model, models.ResNet):
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 10)
    elif isinstance(model, models.VGG):
        num_ftrs = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(num_ftrs, 10)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model = train_model(
        model=model,
        train_loader=cifar10h_loader,
        val_loader=cifar10_val_loader,
        criterion=criterion,
        optimizer=optimizer,
        num_epochs=num_epochs,
    )
    torch.save(model.state_dict(), f"models/{model.__class__.__name__}_cifar10h.pth")

def evaluate_nn_model(model, cifar10_test_loader):
    model.load_state_dict(
        torch.load(f"models/{model.__class__.__name__}_cifar10h.pth", weights_only=True)
    )
    model.eval()

    correct = 0
    total = 0
    device = next(model.parameters()).device
    with torch.no_grad():
        for images, labels in cifar10_test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"{model.__class__.__name__} Accuracy on CIFAR-10 test set: {accuracy:.2f}%")

In [8]:
resnet_model = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
train_nn_model(resnet_model, cifar10h_loader, cifar10_val_loader, lr=0.01)
evaluate_nn_model(resnet_model, cifar10_test_loader)


Training ResNet on CIFAR-10H...
Using device: mps
Epoch [1/20] Train Loss: 1.3566, Validation Loss: 1.7180, Accuracy: 58.77%
Saved model with improved validation accuracy: 58.77%
Epoch [2/20] Train Loss: 0.8915, Validation Loss: 0.8933, Accuracy: 69.05%
Saved model with improved validation accuracy: 69.05%
Epoch [3/20] Train Loss: 0.6604, Validation Loss: 0.8535, Accuracy: 71.59%
Saved model with improved validation accuracy: 71.59%
Epoch [4/20] Train Loss: 0.5384, Validation Loss: 1.1491, Accuracy: 64.92%
Epoch [5/20] Train Loss: 0.4909, Validation Loss: 0.8586, Accuracy: 72.52%
Saved model with improved validation accuracy: 72.52%
Epoch [6/20] Train Loss: 0.3884, Validation Loss: 0.8214, Accuracy: 74.01%
Saved model with improved validation accuracy: 74.01%
Epoch [7/20] Train Loss: 0.3394, Validation Loss: 0.9465, Accuracy: 71.64%
Epoch [8/20] Train Loss: 0.3005, Validation Loss: 0.8234, Accuracy: 75.04%
Saved model with improved validation accuracy: 75.04%
Epoch [9/20] Train Loss: 

In [9]:
vgg_model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
train_nn_model(vgg_model, cifar10h_loader, cifar10_val_loader, lr=0.001)
evaluate_nn_model(vgg_model, cifar10_test_loader)


Training VGG on CIFAR-10H...
Using device: mps
Epoch [1/20] Train Loss: 2.2085, Validation Loss: 1.8863, Accuracy: 18.87%
Saved model with improved validation accuracy: 18.87%
Epoch [2/20] Train Loss: 1.7973, Validation Loss: 2.1571, Accuracy: 25.99%
Saved model with improved validation accuracy: 25.99%
Epoch [3/20] Train Loss: 1.8139, Validation Loss: 1.6829, Accuracy: 31.70%
Saved model with improved validation accuracy: 31.70%
Epoch [4/20] Train Loss: 1.6383, Validation Loss: 1.5933, Accuracy: 30.55%
Epoch [5/20] Train Loss: 1.5082, Validation Loss: 1.5470, Accuracy: 36.31%
Saved model with improved validation accuracy: 36.31%
Epoch [6/20] Train Loss: 1.3558, Validation Loss: 1.2906, Accuracy: 47.25%
Saved model with improved validation accuracy: 47.25%
Epoch [7/20] Train Loss: 1.2409, Validation Loss: 1.3168, Accuracy: 49.29%
Saved model with improved validation accuracy: 49.29%
Epoch [8/20] Train Loss: 1.0533, Validation Loss: 1.0867, Accuracy: 60.81%
Saved model with improved va

## Training Machine Learning Models

In [10]:
def train_ml_models(model, X_cifar10h_scaled, y_cifar10h):
    # Machine Learning models

    print(f"\nTraining {model.__class__.__name__} on CIFAR-10H...")
    model.fit(X_cifar10h_scaled, y_cifar10h)  # Use scaled data


def evaluate_ml_models(model, X_cifar10_scaled, y_cifar10):
    y_pred = model.predict(X_cifar10_scaled)  # Use scaled data
    accuracy = accuracy_score(y_cifar10, y_pred)
    accuracy = 100 * accuracy
    print(f"{model.__class__.__name__} Accuracy on CIFAR-10 test set: {accuracy:.2f}%")

In [11]:
logistic_model = LogisticRegression(max_iter=3000, n_jobs=-1)
train_ml_models(logistic_model, X_cifar10h, y_cifar10h)
evaluate_ml_models(logistic_model, X_cifar10_test, y_cifar10_test)


Training LogisticRegression on CIFAR-10H...
LogisticRegression Accuracy on CIFAR-10 test set: 28.23%


In [12]:
random_forest_model = RandomForestClassifier(n_jobs=-1)
train_ml_models(random_forest_model, X_cifar10h, y_cifar10h)
evaluate_ml_models(random_forest_model, X_cifar10_test, y_cifar10_test)


Training RandomForestClassifier on CIFAR-10H...
RandomForestClassifier Accuracy on CIFAR-10 test set: 41.49%


In [13]:
xgb_model = XGBClassifier(n_jobs=-1)
train_ml_models(xgb_model, X_cifar10h, y_cifar10h)
evaluate_ml_models(xgb_model, X_cifar10_test, y_cifar10_test)


Training XGBClassifier on CIFAR-10H...
XGBClassifier Accuracy on CIFAR-10 test set: 47.35%
